# Solução para o problema no kaggle do titanic

Essa solução faz parte do programa de Iniciação cientifica, a resolução foi proposta pelo meu professor e orientador Rodrigo sanches miane, que possui objetivo de introduzir ao mundo da ciência de dados já que usamos esses conceitos durante minha iniciação ciêntifica.

In [212]:
# As bibliotecas que usaremos

import pandas as PD
from pandas import DataFrame
import matplotlib.pyplot as plt



* Abrindo e analizando os dados.

In [270]:
# abrindo os dados de treino e teste disponibilizados pelo kaggle

data_training = PD.read_csv("train.csv", sep=',', low_memory=False)
data_test = PD.read_csv("test.csv", sep=',', low_memory=False)

data_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


* Gerando relatório para analisar melhor os dados.

In [214]:
# gerando relatório para melhor visualização dos dados
import ydata_profiling

# profile_Normal = ydata_profiling.ProfileReport(data_training, minimal=True)

# profile_Normal.to_file("relatorio_train.html")

In [215]:

# Passo 1 entendnedo os dados procurando por colunas com dados muitos distintos
data_training.nunique().sort_values(ascending=False)

PassengerId    891
Name           891
Ticket         681
Fare           248
Cabin          147
Age             88
SibSp            7
Parch            7
Pclass           3
Embarked         3
Survived         2
Sex              2
dtype: int64

In [216]:
# Dados da variável data_col_erros
data_col_erros = data_training.isna().sum()

data_col_erros = data_col_erros[data_col_erros > 0]

# Obtendo os nomes das colunas
colunas = data_col_erros.index.tolist()

# Obtendo os valores dos erros
erros = data_col_erros.values.tolist()

# Plotando o gráfico de barras
plt.bar(colunas, erros)

# Adicionando rótulos aos eixos x e y
plt.xlabel('Colunas')
plt.ylabel('Quantidade de Erros')

# Adicionando um título ao gráfico
plt.title('Erros por Coluna')

# Rotacionando os rótulos do eixo x para facilitar a leitura
plt.xticks(rotation=90)

# Exibindo o gráfico
plt.show()

/tmp/ipykernel_9906/1832041576.py:26: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [217]:
# analizano as informações de cada coluna como tipo 

data_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


* Como vimos temos muito pouco valores não nulos na coluna Cabine estão os dados dela não será tratado.
* a coluna idade nós a preencheremos com a média das idades.

In [218]:
# substituindo os valores faltantes da mediana
mean_age_train = data_training['Age'].mean()
mean_age_test = data_test['Age'].mean()

data_training['Age'] = data_training['Age'].fillna(value=mean_age_train)
data_test['Age'] = data_test['Age'].fillna(value=mean_age_test)

* Agora umas colunas que são do tipo String que transformaremos em uma representação numerica para se enquadrar no nosso algoritmo de classificação 

In [219]:
#Um valor da coluna fire dos dados de teste esta com valor nan então vamos subistiruir pela moda
mode_fire_test = float(data_test['Fare'].mode())
data_test['Fare'] = data_test['Fare'].fillna(value=mode_fire_test)

data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [220]:
## Substituir os valores na coluna 'Sex' 0 para macho e 1 para female
data_training['Sex'] = data_training['Sex'].map({'male': 0, 'female': 1})
data_test['Sex'] = data_test['Sex'].map({'male':0,'female':1})

# Verificar os valores únicos na coluna 'Sex' se a troca realmente aconteceu
unique_values_traning = data_training['Sex'].unique()
unique_values_test = data_test['Sex'].unique()

# Trocar o tipo da coluna 'Sex' para int
data_training['Sex'] = data_training['Sex'].astype(int)
data_test['Sex'] = data_test['Sex'].astype(int)

data_test.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

# Faze 3
* pegaremos as colunas que não são do tipo numerico que possuem muitos valores diferentes e buscaremos uma forma de representa-las em valores numéricos.
* já as que possuem valores distintos podem confundir noddo modelo então deixaremos assim.

In [221]:
# obtendo as colunas que 
data_training.nunique().sort_values()

Survived         2
Sex              2
Pclass           3
Embarked         3
SibSp            7
Parch            7
Age             89
Cabin          147
Fare           248
Ticket         681
PassengerId    891
Name           891
dtype: int64

* Antes de antes de mudar a coluna para uma representação numética vamos preencaher alguns dados faltantes para que nossos dados de treinamento fique o mais completo possivel.

In [222]:
# obtendo a moda da coluna
mode_embarked_train = str(data_training['Embarked'].mode())
mode_embarked_test = str(data_test['Embarked'].mode())

#preenchendo os nullos pela moda
data_training['Embarked'] = data_training['Embarked'].fillna(value=mode_embarked_train)
data_test['Embarked'] = data_test['Embarked'].fillna(value=mode_embarked_test)

#visualizando os nullos e os tipos.
data_training.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


* Agora com auxilio do OndeHotEncoder valos representar os valores da coluna embarked para numerico ou seja dentro das três possibilidades S (Southampton), C (Cherbourg) e Q (Queenstown) onde tiver o valor 1 indica o local de embarque do passageiro.

In [223]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Codificação one-hot da coluna 'Embarked' no DataFrame de treinamento
encoder = OneHotEncoder()
embarked_encoded_train = pd.DataFrame(encoder.fit_transform(data_training[['Embarked']]).toarray(), columns=encoder.categories_[0].tolist())

# Codificação one-hot da coluna 'Embarked' no DataFrame de teste
embarked_encoded_test = pd.DataFrame(encoder.transform(data_test[['Embarked']]).toarray(), columns=encoder.categories_[0].tolist())

# Removendo a coluna 'Embarked'
data_training = data_training.drop(columns=['Embarked'])
data_test = data_test.drop(columns=['Embarked'])

#removando coluna extra que gerou
subset_train = embarked_encoded_train.loc[:, ['C', 'Q', 'S']]
subset_test = embarked_encoded_test.loc[:, ['C', 'Q', 'S']]

# Concatenando o DataFrame original com a nova coluna codificada
# data_training = pd.concat([data_training, subset_train], axis=1)
# data_test = pd.concat([data_test, subset_test], axis=1)

data_training = data_training.join(subset_train)
data_test = data_test.join(subset_test)

data_training


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,C,Q,S
0,1,0,3,"Braund, Mr. Owen Harris",0,22.000000,1,0,A/5 21171,7.2500,NaN,0.0,0.0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.000000,1,0,PC 17599,71.2833,C85,1.0,0.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,0.0,0.0,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.000000,1,0,113803,53.1000,C123,0.0,0.0,1.0
4,5,0,3,"Allen, Mr. William Henry",0,35.000000,0,0,373450,8.0500,NaN,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,27.000000,0,0,211536,13.0000,NaN,0.0,0.0,1.0
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.000000,0,0,112053,30.0000,B42,0.0,0.0,1.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,29.699118,1,2,W./C. 6607,23.4500,NaN,0.0,0.0,1.0
889,890,1,1,"Behr, Mr. Karl Howell",0,26.000000,0,0,111369,30.0000,C148,1.0,0.0,0.0


# Faze 4
* Removendo colunas que possuiem muitos valores distintos e seus tipos não são numericos.

In [224]:
# selecionando todas as colunas que os tipos são diferentes de numericos e removendo
index_numbers_train = data_training.select_dtypes(include=['number']).columns
data_training_bin = data_training[index_numbers_train]

# selecionando todas as colunas que o tipos são diferentes de numeric e removendo
index_numbers_test = data_test.select_dtypes(include=['number']).columns
data_test_bin = data_test[index_numbers_test]

# verificando as colunas são as mesmas após a remoção
data_training_bin.columns.difference(data_test_bin.columns)

Index(['Survived'], dtype='object')

In [225]:
data_test_bin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    int64  
 3   Age          418 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Fare         418 non-null    float64
 7   C            418 non-null    float64
 8   Q            418 non-null    float64
 9   S            418 non-null    float64
dtypes: float64(5), int64(5)
memory usage: 32.8 KB


In [226]:
types = data_test_bin.dtypes
types2 = data_training_bin.dtypes

types.index.difference(types2.index)

Index([], dtype='object')

In [227]:
data_training_bin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   C            891 non-null    float64
 9   Q            891 non-null    float64
 10  S            891 non-null    float64
dtypes: float64(5), int64(6)
memory usage: 76.7 KB


### Agora podemos treinar nosso modelo já que todos as colunas são numericas e não existe nenhum vaor null
* Separandos o vetor de treino e teste

In [228]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import random

random.randrange(10, 40) / 100

training_y = np.array(data_training_bin['Survived'])

training_x = np.array(data_training_bin.drop(columns=['Survived', 'PassengerId']))

test_x = np.array(data_test_bin.drop(columns=['PassengerId']))

teste_target = PD.read_csv("gender_submission.csv")

teste_y = np.array(teste_target['Survived'])

data_training_bin

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
0,1,0,3,0,22.000000,1,0,7.2500,0.0,0.0,1.0
1,2,1,1,1,38.000000,1,0,71.2833,1.0,0.0,0.0
2,3,1,3,1,26.000000,0,0,7.9250,0.0,0.0,1.0
3,4,1,1,1,35.000000,1,0,53.1000,0.0,0.0,1.0
4,5,0,3,0,35.000000,0,0,8.0500,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.000000,0,0,13.0000,0.0,0.0,1.0
887,888,1,1,1,19.000000,0,0,30.0000,0.0,0.0,1.0
888,889,0,3,1,29.699118,1,2,23.4500,0.0,0.0,1.0
889,890,1,1,0,26.000000,0,0,30.0000,1.0,0.0,0.0


* Vamos treinar nossos modelos vamos começar com arvores de desisão

In [229]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import numpy as np

# Criando um objeto DecisionTreeClassifier
clf = DecisionTreeClassifier()

# Treinando o classificador
clf.fit(training_x, training_y)

# Fazendo previsões no conjunto de teste
y_tree = clf.predict(test_x)

# Avaliando a precisão das previsões
accuracy = accuracy_score(teste_y, y_tree)
print("Acurácia:", accuracy)

Acurácia: 0.777511961722488


In [230]:
confusion_matrix(teste_y, y_tree)

array([[220,  46],
       [ 47, 105]])

In [231]:
tree = PD.Series(y_tree, index=teste_target['PassengerId'], name='Survived')

tree.to_csv("rfc.csv")

* Agora vamos ver o resultado em uma Floresta aletória.

In [232]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(n_estimators=500, n_jobs=-1, max_leaf_nodes=16, random_state=0)

RFC.fit(training_x, training_y)

y_rfc = RFC.predict(test_x)

accuracy = accuracy_score(teste_y, y_rfc)

print("Acurácia:", accuracy)

Acurácia: 0.9114832535885168


In [233]:
confusion_matrix(teste_y, y_rfc)

array([[257,   9],
       [ 28, 124]])

In [234]:
saida_rfc = PD.Series(y_rfc, index=teste_target['PassengerId'], name='Survived')

saida_rfc.to_csv("rfc.csv")

* Testando Regressão logistica.

In [235]:
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression()

log_clf.fit(training_x, training_y)

y_log_clf = log_clf.predict(test_x)

accuracy = accuracy_score(teste_y, y_log_clf)

print('Accuracy: ', accuracy)

Accuracy:  0.9449760765550239


/home/maycon/IC-RedesDomesticas/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [236]:
confusion_matrix(teste_y, y_log_clf)

array([[253,  13],
       [ 10, 142]])

In [237]:
regLin = pd.Series(y_log_clf, index=teste_target['PassengerId'], name='Survived')

regLin.to_csv("logLin.csv")

In [238]:
from sklearn.svm import SVC

svc = SVC()

svc.fit(training_x, training_y)

y_svc = svc.predict(test_x)

accuracy = accuracy_score(teste_y, y_svc)

print("Accuracy: ", accuracy)

Accuracy:  0.6555023923444976


In [239]:
confusion_matrix(teste_y, y_svc)

array([[232,  34],
       [110,  42]])

In [240]:
from sklearn.ensemble import BaggingClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=6,
    bootstrap=True,
    n_jobs=-1
)


bag_clf.fit(training_x, training_y)

y_bag = bag_clf.predict(test_x)

accuracy = accuracy_score(teste_y, y_bag)

print("Accuracy: ", accuracy)

Accuracy:  0.8564593301435407


In [241]:
confusion_matrix(teste_y, y_bag)

array([[264,   2],
       [ 58,  94]])

In [242]:
bag = pd.Series(y_bag, index=teste_target['PassengerId'], name='Survived')

bag.to_csv("bag.csv")

In [243]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=5)

KNN.fit(training_x, training_y)

y_pre_knn = KNN.predict(test_x)

accuracy = accuracy_score(teste_y, y_pre_knn)

print("acurácia: ", accuracy)

acurácia:  0.6722488038277512


In [244]:
confusion_matrix(teste_y, y_pre_knn)

array([[200,  66],
       [ 71,  81]])

In [262]:
erros = teste_target

erros['y_Tree'] =  y_tree
erros['y_RandomForestClf'] = y_rfc
erros['y_LogisticReqresion'] = y_log_clf
erros['y_BaggingClassifier'] = y_bag
erros['y_KNN'] = y_pre_knn

tests = pd.read_csv('./test.csv')

ret = erros[erros['y_RandomForestClf'] != erros['Survived']]

erros_RFC = tests[tests['PassengerId'].isin(ret['PassengerId'])]

erros_RFC.to_csv("erros.csv")

erros_RFC.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.000,NaN,S
18,910,3,"Ilmakangas, Miss. Ida Livija",female,27.0,1,0,STON/O2. 3101270,7.925,NaN,S
19,911,3,"Assaf Khalil, Mrs. Mariana (Miriam"")""",female,45.0,0,0,2696,7.225,NaN,C
32,924,3,"Dean, Mrs. Bertram (Eva Georgetta Light)",female,33.0,1,2,C.A. 2315,20.575,NaN,S
33,925,3,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",female,NaN,1,2,W./C. 6607,23.450,NaN,S


In [263]:
import ydata_profiling

profile_Normal = ydata_profiling.ProfileReport(erros_RFC, minimal=True)

profile_Normal.to_file("erros_RFC.html")

/home/maycon/IC-RedesDomesticas/.env/lib/python3.10/site-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 280.93it/s]


In [264]:
erros_RFC['Pclass'].value_counts()

3    33
1     2
2     2
Name: Pclass, dtype: int64